In [1]:
import os
import csv
import sklearn
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.linear_model as lm 

from pylab import rcParams
from sklearn import metrics 
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.cluster import KMeans
from pandas import Series, DataFrame
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier 
from sklearn.preprocessing import StandardScaler  
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
file_dir = os.path.join('/Users/digisha/Documents/DataScienceWithPython/Digisha_Singhal_assignment12')
file_name = os.path.join(file_dir, 'Sprint.csv')
df = pd.read_csv(file_name)
df.rename(columns={'Adj Close': 'price'}, inplace = True)
df['pct_change'] = df['price'].pct_change()
#df['std'] = df['pct_change'].rolling(5).std()
df['label'] = (df['pct_change'] > 0).replace((True,False), (1,0))
df.head()

,Date,Open,High,Low,Close,price,Volume,pct_change,label
0,2013-09-19,6.46,6.50,6.30,6.49,6.49,13184600,NaN,0
1,2013-09-20,6.48,6.49,6.18,6.26,6.26,28682200,-0.035439,0
2,2013-09-23,6.19,6.20,6.02,6.15,6.15,30035400,-0.017572,0
3,2013-09-24,6.14,6.24,6.05,6.19,6.19,12635500,0.006504,1
4,2013-09-25,6.28,6.30,6.04,6.19,6.19,19995500,0.000000,0


In [3]:
df['Date'] = df['Date'].astype(str)
df['Date'] = df['Date'].str.replace('-', '')
df['Date'] = pd.to_datetime(df['Date'], format="%Y-%m-%d")
dayOfWeek={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
df['weekday'] = df['Date'].dt.dayofweek.map(dayOfWeek)

k = 0
df['return_fri_mon']=''
df['monday_open']=''
for i in range(0,len(df)-1):
    k = i+1
    #for fri to mon day returns, when other days would be dropped
    df['return_fri_mon'].loc[i] = df['price'].pct_change().loc[k]
    #for mon opeing price when other days will be dropped
    df['monday_open'].loc[i] = df['price'].loc[k]
    
df.head()

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Date,Open,High,Low,Close,price,Volume,pct_change,label,weekday,return_fri_mon,monday_open
0,2013-09-19,6.46,6.50,6.30,6.49,6.49,13184600,NaN,0,Thursday,-0.0354391,6.26
1,2013-09-20,6.48,6.49,6.18,6.26,6.26,28682200,-0.035439,0,Friday,-0.0175719,6.15
2,2013-09-23,6.19,6.20,6.02,6.15,6.15,30035400,-0.017572,0,Monday,0.00650407,6.19
3,2013-09-24,6.14,6.24,6.05,6.19,6.19,12635500,0.006504,1,Tuesday,0,6.19
4,2013-09-25,6.28,6.30,6.04,6.19,6.19,19995500,0.000000,0,Wednesday,0.0177706,6.3


In [4]:
ndf = df
df['r1'] = '' #daily return vector for monday
df['l1'] = ''#label vector for monday

df['r2'] = ''
df['l2'] = ''

df['r3'] = ''
df['l3'] = ''

df['r4'] = ''
df['l4'] = ''

df['r5'] = ''
df['l5'] = ''

df['true_range'] = ''

for day in range(0,len(ndf)-4):
    if ndf['weekday'].loc[day] == 'Monday':
        ndf['r1'].loc[day] = ndf['pct_change'].loc[day]
        ndf['r2'].loc[day] = ndf['pct_change'].loc[day + 1]
        ndf['r3'].loc[day] = ndf['pct_change'].loc[day + 2]
        ndf['r4'].loc[day] = ndf['pct_change'].loc[day + 3]
        ndf['r5'].loc[day] = ndf['pct_change'].loc[day + 4]
        
        ndf['l1'].loc[day] = ndf['label'].loc[day]
        ndf['l2'].loc[day] = ndf['label'].loc[day + 1]
        ndf['l3'].loc[day] = ndf['label'].loc[day + 2]
        ndf['l4'].loc[day] = ndf['label'].loc[day + 3]
        ndf['l5'].loc[day] = ndf['label'].loc[day + 4]
        ndf['price'].loc[day] = ndf['price'].loc[day + 4] #taking this week's friday closing price
    
        continue
    else:
        ndf = ndf.drop(day)

ndf = ndf.reset_index(drop=True)

for day in range(1,len(ndf)):
    ndf['true_range'].loc[day] = (ndf['price'].loc[day] - ndf['price'].loc[day-1])

ndf.fillna(0, inplace = True)        
ndf = ndf.reset_index(drop=True)
ndf.head()

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Date,Open,High,Low,Close,price,Volume,pct_change,label,weekday,...,l1,r2,l2,r3,l3,r4,l4,r5,l5,true_range
0,2013-09-23,6.19,6.20,6.02,6.15,6.16,30035400,-0.017572,0,Monday,...,0,0.00650407,1,0,0,0.0177706,1,-0.0222222,0,
1,2013-09-30,6.09,6.27,6.04,6.22,6.44,19916300,0.009740,1,Monday,...,1,0.011254,1,0.00953895,1,-0.00944882,0,0.0238474,1,0.28
2,2013-10-07,6.41,6.45,6.36,6.38,6.00,6762500,-0.009317,0,Monday,...,0,-0.039185,0,-0.0212072,0,-0.005,0,0.00502513,1,-0.44
3,2013-10-14,6.00,6.09,6.00,6.03,6.40,8172900,0.005000,1,Monday,...,1,0,0,0.0447761,1,0.0190476,1,-0.00311526,0,0.4
4,2013-10-21,6.45,6.48,6.33,6.35,6.45,5440100,-0.007813,0,Monday,...,0,0.00629921,1,0.00625978,1,0.0139969,1,-0.0107362,0,0.05


In [5]:
ndf['weekly_pct'] = ndf['price'].pct_change()
ndf['weekly_label'] =(ndf['weekly_pct'] > 0).replace((True,False), (1,0))
ndf.head()

,Date,Open,High,Low,Close,price,Volume,pct_change,label,weekday,...,l2,r3,l3,r4,l4,r5,l5,true_range,weekly_pct,weekly_label
0,2013-09-23,6.19,6.20,6.02,6.15,6.16,30035400,-0.017572,0,Monday,...,1,0,0,0.0177706,1,-0.0222222,0,,NaN,0
1,2013-09-30,6.09,6.27,6.04,6.22,6.44,19916300,0.009740,1,Monday,...,1,0.00953895,1,-0.00944882,0,0.0238474,1,0.28,0.045455,1
2,2013-10-07,6.41,6.45,6.36,6.38,6.00,6762500,-0.009317,0,Monday,...,0,-0.0212072,0,-0.005,0,0.00502513,1,-0.44,-0.068323,0
3,2013-10-14,6.00,6.09,6.00,6.03,6.40,8172900,0.005000,1,Monday,...,0,0.0447761,1,0.0190476,1,-0.00311526,0,0.4,0.066667,1
4,2013-10-21,6.45,6.48,6.33,6.35,6.45,5440100,-0.007813,0,Monday,...,1,0.00625978,1,0.0139969,1,-0.0107362,0,0.05,0.007812,1


In [6]:
mask = (ndf['Date'] >= '2017-01-01') & (ndf['Date'] < '2018-01-01')
#print(df.loc[mask])
new1_df = ndf[['Date','weekly_label', 'price','r1', 'l1', 'r2', 'l2', 'r3', 'l3', 'r4', 'l4','r5', 'l5']]
new1_df = new1_df.loc[mask] #4 weeks of jan
new1_df = new1_df.reset_index(drop=True)
new1_df.tail()

,Date,weekly_label,price,r1,l1,r2,l2,r3,l3,r4,l4,r5,l5
41,2017-11-20,0,6.15,-0.00321027,0,-0.020934,0,0.0115132,1,0,0,0,0
42,2017-11-27,0,5.97,0,0,-0.0130081,0,0,0,-0.0131796,0,-0.0033389,0
43,2017-12-04,0,5.44,-0.00335008,0,-0.0352941,0,-0.0087108,0,-0.0404218,0,-0.003663,0
44,2017-12-11,1,5.49,0,0,0.0441176,1,0.00176056,1,-0.0105448,0,-0.0248668,0
45,2017-12-18,1,5.64,0.0200364,1,-0.00357143,0,0,0,0.00716846,1,0.00355872,1


In [7]:
mask = (ndf['Date'] >= '2018-01-01') & (ndf['Date'] < '2018-09-01')
#print(df.loc[mask])
new_df = ndf[['Date','weekly_label', 'price','r1', 'l1', 'r2', 'l2', 'r3', 'l3', 'r4', 'l4','r5', 'l5']]
new_df = new_df.loc[mask] #4 weeks of jan
new_df = new_df.reset_index(drop=True)
new_df.tail()

,Date,weekly_label,price,r1,l1,r2,l2,r3,l3,r4,l4,r5,l5
26,2018-07-30,1,5.62,0,0,-0.010929,0,-0.0092081,0,0.0520446,1,-0.00706714,0
27,2018-08-06,1,6.03,0.0996441,1,-0.00809061,0,-0.00815661,0,0,0,-0.00822368,0
28,2018-08-13,1,6.13,0.0066335,1,0.00658979,1,-0.0163666,0,0.0183028,1,0.00163399,1
29,2018-08-20,0,6.08,0.00652529,1,0.00162075,1,-0.00647249,0,-0.00488599,0,-0.00490998,0
30,2018-08-27,1,6.11,0.00328947,1,0,0,0.00491803,1,0,0,-0.00326264,0


In [8]:
newdf = new_df.loc[0:19, :]
#newdf.tail(22)
len(newdf) #first 20 weeks for 2018

20

In [9]:
#classification 1 - Logistic regression
def class1():
    rcParams['figure.figsize'] = 10, 8
    sns.set_style('whitegrid')
    
    #train data
    X1 = new1_df.ix[:,(3,4,5)].values
    y1 = new1_df.ix[:,1].values
    X_train, X_test_1, y_train, y_test_1 = train_test_split(X1, y1, test_size = 0.4, random_state=100)
    
    #test data
    X_test = newdf.ix[:,(3,4,5)].values
    y_test = newdf.ix[:,1].values
    
    #X_train_1, X_test, y_train_1, y_test = train_test_split(X, y, test_size = 0.5, random_state=100)

    # Logistic Regression : 
    LogReg = LogisticRegression()
    # Fit the regressor to the training data
    LogReg.fit(X_train, y_train)

    y_pred = LogReg.predict(X_test)
    #print(y_test)
    #print(y_pred)

    correct_1 = 0
    incorrect_1 = 0
    correct_0 = 0
    incorrect_0 = 0
    total_0 = 0
    total_1 = 0

    for i in range(0,len(y_pred)):
        if y_pred[i] == 1:
            total_1 += 1
        else:
            total_0 += 1

        if y_test[i] == 1 and y_test[i] == y_pred[i]:
            correct_1 += 1
        elif y_test[i] == 1 and y_test[i] != y_pred[i]:
            incorrect_1 += 1
        elif y_test[i] == 0 and y_test[i] != y_pred[i]:
            incorrect_0 += 1
        else:
            correct_0 += 1
    #print(correct_1, incorrect_1, incorrect_0, correct_0)
    if total_0 != 0 and total_1 != 0:
        per_1 = (correct_1/total_1)*100
        per_0 = (correct_0/total_0)*100
    else:
        per_1 = 0
        per_0 = 0
    #print(per_1, total_1, per_0, total_0)

    return per_1, total_1, per_0, total_0

a1, b1, c1, d1 = class1()
print(a1, b1, c1, d1)

0 20 0 0


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: 
.ix

In [10]:
#classification 2 - KNN (k = 5)
def class2(k):
    data_feature_names = ['l1', 'l2','l3', 'l4', 'l5']
    
    #train data
    X1 = new1_df.ix[:,(3,4,5)].values
    y1 = new1_df.ix[:,1].values
    X_train, X_test_1, y_train, y_test_1 = train_test_split(X1, y1, test_size = 0.4, random_state=100)
    
    #test data
    X_test = newdf.ix[:,(3,4,5)].values
    y_test = newdf.ix[:,1].values
    #X_train_1, X_test, y_train_1, y_test = train_test_split(X, y, test_size = 0.5, random_state=100)
  
    scaler = StandardScaler()  
    scaler.fit(X_train)

    X_train = scaler.transform(X_train)  
    X_test = scaler.transform(X_test)


    
    classifier = KNeighborsClassifier(n_neighbors=k)  
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)
    
    correct_1 = 0
    incorrect_1 = 0
    correct_0 = 0
    incorrect_0 = 0
    total_0 = 0
    total_1 = 0
    for i in range(0,len(y_pred)):
        if y_pred[i] == 1:
            total_1 += 1
        else:
            total_0 += 1

        if y_test[i] == 1 and y_test[i] == y_pred[i]:
            correct_1 += 1
        elif y_test[i] == 1 and y_test[i] != y_pred[i]:
            incorrect_1 += 1
        elif y_test[i] == 0 and y_test[i] != y_pred[i]:
            incorrect_0 += 1
        else:
            correct_0 += 1
    #print(correct_1, incorrect_1, incorrect_0, correct_0)
    if total_0 != 0 and total_1 != 0:
        per_1 = (correct_1/total_1)*100
        per_0 = (correct_0/total_0)*100
    else:
        per_1 = 0
        per_0 = 0

    #print(per_1, total_1, per_0, total_0)
    return per_1, total_1, per_0, total_0
  
a2, b2, c2, d2 = class2(5)
print(a2, b2, c2, d2)

55.55555555555556 9 45.45454545454545 11


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/lib/python3.6/site-packages/ipykernel_launch

In [11]:
#classification 3 - Naïve Bayesian 
def class3():    
    data_feature_names = ['l1', 'l2','l3', 'l4', 'l5']
    
    #train data
    X_train, X_test_1 = train_test_split(new1_df, test_size=0.4, random_state=100)
    
    #test data
    X_test = newdf
    
    #X_train_1, X_test = train_test_split(newdf, test_size=0.5, random_state=100)
    
    #print(X_train, X_test)
    
    gnb = GaussianNB()
    gnb.fit(X_train[data_feature_names].values, X_train["weekly_label"])
    y_pred = gnb.predict(X_test[data_feature_names])
    X_test.loc[:, 'predicted'] = y_pred
    y_test =  X_test.loc[: ,'weekly_label'].values

    #print(y_pred)
    #print(y_test)
    #print(X_test['weekly_label'])
    #print(X_train["weekly_label"])

    correct_1 = 0
    incorrect_1 = 0
    correct_0 = 0
    incorrect_0 = 0
    total_0 = 0
    total_1 = 0
    for i in range(0,len(y_pred)):
        if y_pred[i] == 1:
            total_1 += 1
        else:
            total_0 += 1

        if y_test[i] == 1 and y_test[i] == y_pred[i]:
            correct_1 += 1
        elif y_test[i] == 1 and y_test[i] != y_pred[i]:
            incorrect_1 += 1
        elif y_test[i] == 0 and y_test[i] != y_pred[i]:
            incorrect_0 += 1
        else:
            correct_0 += 1
    #print(correct_1, incorrect_1, incorrect_0, correct_0)
    if total_0 != 0 and total_1 != 0:
        per_1 = (correct_1/total_1)*100
        per_0 = (correct_0/total_0)*100
    else:
        per_1 = 0
        per_0 = 0

    #print(per_1, total_1, per_0, total_0)
    return per_1, total_1, per_0, total_0

a3, b3, c3, d3 = class3()
print(a3, b3, c3, d3)

66.66666666666666 9 54.54545454545454 11


/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [12]:
#classification 4 - kMeans (k = 3)
def class4(k):
    
    #train data
    X1 = new1_df.iloc[:, [4,6,8,10,12]].values
    y1 = new1_df.iloc[:, 1].values
    x_scale = preprocessing.scale(X1)
    X_train, X_test_1, y_train, y_test_1 = train_test_split(X1, y1, test_size = 0.4, random_state=100)
    
    #test data
    X_test = newdf.iloc[:, [4,6,8,10,12]].values
    y_test = newdf.iloc[:, 1].values
    pct = newdf.iloc[:, [2]].values
    #X_train_1, X_test, y_train_1, y_test = train_test_split(X, y, test_size = 0.5, random_state=100)

    #print ('means: ', x_scale.mean(axis=0))
    #print('st dev: ', x_scale.std(axis=0))

    min_max_scaler = preprocessing.MinMaxScaler()
    x_min_max = min_max_scaler.fit_transform(X1)

    km = KMeans(n_clusters=k, init='k-means++',  #with k = 3
                n_init=10, max_iter=300, tol=1e-04,
                random_state=0)

    y_pred = km.fit_predict(X_test)
    #print(y_pred)
    #print(y_test)

    correct_1 = 0
    incorrect_1 = 0
    correct_0 = 0
    incorrect_0 = 0
    total_0 = 0
    total_1 = 0
    for i in range(0,len(y_pred)):
        if y_pred[i] == 1:
            total_1 += 1
        else:
            total_0 += 1

        if y_test[i] == 1 and y_test[i] == y_pred[i]:
            correct_1 += 1
        elif y_test[i] == 1 and y_test[i] != y_pred[i]:
            incorrect_1 += 1
        elif y_test[i] == 0 and y_test[i] != y_pred[i]:
            incorrect_0 += 1
        else:
            correct_0 += 1
    #print(correct_1, incorrect_1, incorrect_0, correct_0)
    if total_0 != 0 and total_1 != 0:
        per_1 = (correct_1/total_1)*100
        per_0 = (correct_0/total_0)*100
    else:
        per_1 = 0
        per_0 = 0

    #print(per_1, total_1, per_0, total_0)
    return per_1, total_1, per_0, total_0

a4, b4, c4, d4 = class4(3)  #k = 3
print(a4, b4, c4, d4)

37.5 8 25.0 12


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [13]:
#classification 5 - Linear Regression
def class5():
    data_feature_names = ['l1', 'l2','l3', 'l4', 'l5']
    
    X1 = new1_df[data_feature_names].values
    y1 = new1_df['weekly_label'].values
    
    #train data
    X_train, X_test_1, y_train, y_test_1 = train_test_split(X1, y1, test_size=0.4, random_state=100)
    
    #test data
    X_test = newdf[data_feature_names].values
    y_test = newdf['weekly_label'].values
    
    linreg = LinearRegression()
    linreg.fit(X_train,y_train)

    y_pred = linreg.predict(X_test)

    for i in range(0,len(y_pred)):
        #print(i)
        if y_pred[i] > 0.5:
            y_pred[i] = 1.00
        else:
            y_pred[i] = 0  

    #print(y_pred)
    #print(y_test)

    correct_1 = 0
    incorrect_1 = 0
    correct_0 = 0
    incorrect_0 = 0
    total_0 = 0
    total_1 = 0
    for i in range(0,len(y_pred)):
        if y_pred[i] == 1:
            total_1 += 1
        else:
            total_0 += 1

        if y_test[i] == 1 and y_test[i] == y_pred[i]:
            correct_1 += 1
        elif y_test[i] == 1 and y_test[i] != y_pred[i]:
            incorrect_1 += 1
        elif y_test[i] == 0 and y_test[i] != y_pred[i]:
            incorrect_0 += 1
        else:
            correct_0 += 1
    #print(correct_1, incorrect_1, incorrect_0, correct_0)
    if total_0 != 0 and total_1 != 0:
        per_1 = (correct_1/total_1)*100
        per_0 = (correct_0/total_0)*100
    else:
        per_1 = 0
        per_0 = 0

    #print(per_1, total_1, per_0, total_0)
    return per_1, total_1, per_0, total_0

a5, b5, c5, d5 = class5()
print(a5, b5, c5, d5)

87.5 8 66.66666666666666 12


In [14]:
#classification 6 - decision tree (using entropy)
def class6(): 
    data_feature_names = ['l1', 'l2','l3', 'l4', 'l5']
    
    X1 = new1_df[data_feature_names].values
    y1 = new1_df['weekly_label'].values
    
    #train data
    X_train, X_test_1, y_train, y_test_1 = train_test_split(X1, y1, test_size=0.4, random_state=100)
    
    #test data
    X_test = newdf[data_feature_names].values
    y_test = newdf['weekly_label'].values
    
    # Creating the classifier object 
    clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100,max_depth=3, min_samples_leaf=5) 

    # Performing training 
    clf_gini.fit(X_train, y_train) 

    clf_entropy = DecisionTreeClassifier(criterion='entropy', max_features=3)

    # Performing training 
    clf_entropy.fit(X_train, y_train) 

    y_pred = clf_entropy.predict(X_test) 
    #print("Predicted values:") 
    #print(y_test)
    #print(y_pred) 

    correct_1 = 0
    incorrect_1 = 0
    correct_0 = 0
    incorrect_0 = 0
    total_0 = 0
    total_1 = 0
    for i in range(0,len(y_pred)):
        if y_pred[i] == 1:
            total_1 += 1
        else:
            total_0 += 1

        if y_test[i] == 1 and y_test[i] == y_pred[i]:
            correct_1 += 1
        elif y_test[i] == 1 and y_test[i] != y_pred[i]:
            incorrect_1 += 1
        elif y_test[i] == 0 and y_test[i] != y_pred[i]:
            incorrect_0 += 1
        else:
            correct_0 += 1
    #print(correct_1, incorrect_1, incorrect_0, correct_0)
    if total_0 != 0 and total_1 != 0:
        per_1 = (correct_1/total_1)*100
        per_0 = (correct_0/total_0)*100
    else:
        per_1 = 0
        per_0 = 0
    
    return per_1, total_1, per_0, total_0

a6, b6, c6, d6 = class6()
print(a6, b6, c6, d6)

70.0 10 60.0 10


In [15]:
#classification 7 - SVM : a) linear, b) radial basis function (Gaussian)
def class7(t):
    
    data_feature_names = ['l1', 'l2','l3', 'l4', 'l5']
    
    X1 = new1_df[data_feature_names].values
    y1 = new1_df['weekly_label'].values
    
    #train data
    X_train, X_test_1, y_train, y_test_1 = train_test_split(X1, y1, test_size=0.4, random_state=100)
    
    #test data
    X = newdf[data_feature_names].values
    y = newdf['weekly_label'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.50, random_state=100)
    
    if t==1:
        svclassifier = SVC(kernel='linear') 
    else:
        svclassifier = SVC(kernel = 'rbf')
        
    svclassifier.fit(X_train, y_train) 
    y_pred = svclassifier.predict(X_test)
    #print(y_pred)
    #print(y_test)
    correct_1 = 0
    incorrect_1 = 0
    correct_0 = 0
    incorrect_0 = 0
    total_0 = 0
    total_1 = 0
    for i in range(0,len(y_pred)):
        if y_pred[i] == 1:
            total_1 += 1
        else:
            total_0 += 1

        if y_test[i] == 1 and y_test[i] == y_pred[i]:
            correct_1 += 1
        elif y_test[i] == 1 and y_test[i] != y_pred[i]:
            incorrect_1 += 1
        elif y_test[i] == 0 and y_test[i] != y_pred[i]:
            incorrect_0 += 1
        else:
            correct_0 += 1
    #print(correct_1, incorrect_1, incorrect_0, correct_0)
    if total_0 != 0 and total_1 != 0:
        per_1 = (correct_1/total_1)*100
        per_0 = (correct_0/total_0)*100
    else:
        per_1 = 0
        per_0 = 0
    
    return per_1, total_1, per_0, total_0

a7, b7, c7, d7 = class7(1) #for linear svm
a8, b8, c8, d8 = class7(0) #for linear svm
print(a7, b7, c7, d7)
print(a8, b8, c8, d8)
#class7(1)

42.857142857142854 7 0.0 3
33.33333333333333 6 0.0 4


In [16]:
df_algo = newdf[['Date','price','weekly_label']]
df_algo['predicted_label'] = df_algo['weekly_label'].shift(1)
df_algo.fillna(0, inplace = True)

#Stratergy 1 - Naive Stratergy
def strat1():

    y_test = df_algo.ix[:,2].values
    y_pred = df_algo.ix[:,3].values
    #print(y_test)
    #print(y_pred)

    correct_1 = 0
    incorrect_1 = 0
    correct_0 = 0
    incorrect_0 = 0
    total_0 = 0
    total_1 = 0
    for i in range(0,len(y_pred)):
        if y_pred[i] == 1:
            total_1 += 1
        else:
            total_0 += 1

        if y_test[i] == 1 and y_test[i] == y_pred[i]:
            correct_1 += 1
        elif y_test[i] == 1 and y_test[i] != y_pred[i]:
            incorrect_1 += 1
        elif y_test[i] == 0 and y_test[i] != y_pred[i]:
            incorrect_0 += 1
        else:
            correct_0 += 1
    #print(correct_1, incorrect_1, incorrect_0, correct_0)
    if total_0 != 0 and total_1 != 0:
        per_1 = (correct_1/total_1)*100
        per_0 = (correct_0/total_0)*100
    else:
        per_1 = 0
        per_0 = 0

    #print(per_1, total_1, per_0, total_0)
    return per_1, total_1, per_0, total_0

a9, b9, c9, d9 = strat1()
print(a9, b9, c9, d9)

60.0 10 50.0 10


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.

In [17]:
df_algo = newdf[['Date','price','weekly_label']] #reset df_algo
#Stratergy 2 - Moving average
def strat2(W):   
    #W = 10
    df_algo['ma'] = df_algo['price'].rolling(W).mean()
    df_algo.fillna(0, inplace = True)
    df_algo['predicted_label'] = 0
    #df_algo.head()

    for i in range(W-1,len(df_algo)):
            #print(i)
            if df_algo['price'].loc[i] > df_algo['ma'].loc[i]:
                df_algo['predicted_label'].loc[i] = 1
            elif df_algo['price'].loc[i] < df_algo['ma'].loc[i]:
                df_algo['predicted_label'].loc[i] = 0
            else:
                df_algo['predicted_label'].loc[i] = df_algo['weekly_label'].loc[i-1]

    y_test = df_algo.ix[:,2].values
    y_pred = df_algo.ix[:,4].values
    #print(y_test)
    #print(y_pred)

    correct_1 = 0
    incorrect_1 = 0
    correct_0 = 0
    incorrect_0 = 0
    total_0 = 0
    total_1 = 0
    for i in range(0,len(y_pred)):
        if y_pred[i] == 1:
            total_1 += 1
        else:
            total_0 += 1

        if y_test[i] == 1 and y_test[i] == y_pred[i]:
            correct_1 += 1
        elif y_test[i] == 1 and y_test[i] != y_pred[i]:
            incorrect_1 += 1
        elif y_test[i] == 0 and y_test[i] != y_pred[i]:
            incorrect_0 += 1
        else:
            correct_0 += 1
    #print(correct_1, incorrect_1, incorrect_0, correct_0)
    if total_0 != 0 and total_1 != 0:
        per_1 = (correct_1/total_1)*100
        per_0 = (correct_0/total_0)*100
    else:
        per_1 = 0
        per_0 = 0

    #print(per_1, total_1, per_0, total_0)
    return per_1, total_1, per_0, total_0

a10, b10, c10, d10 = strat2(10) #W = 10 
print(a10, b10, c10, d10)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


100.0 4 56.25 16


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [18]:
df_algo = newdf[['Date','price','weekly_label']] #reset df_algo
#Stratergy 3 - Bollinger bands stratergy

def strat3(k, W):
    df_algo['st_ma'] = df_algo['price'].rolling(W).mean()
    df_algo['std'] = df_algo['price'].rolling(W).std()

    df_algo['low_band'] = df_algo['st_ma'] - k*df_algo['std']
    df_algo['upper_band'] = df_algo['st_ma'] + k*df_algo['std']

    df_algo['predicted_label'] = 0
    #df_algo.tail()
#assuming if the price> upperband, we buy i.e., expect the price to go up, so predicted label will be 1
# similarily if less than lower band, label = 0
#else no change 
    for i in range(W-1,len(df_algo)):
        #print(i)
        if df_algo['price'].loc[i] > df_algo['upper_band'].loc[i]:
            df_algo['predicted_label'].loc[i] = 1
        elif df_algo['price'].loc[i] < df_algo['low_band'].loc[i]:
            df_algo['predicted_label'].loc[i] = 0
        else:
            df_algo['predicted_label'].loc[i] = df_algo['weekly_label'].loc[i-1]

    #df_algo.tail()

    y_test = df_algo.ix[:,2].values
    y_pred = df_algo.ix[:,7].values
    #print(y_test)
    #print(y_pred)

    correct_1 = 0
    incorrect_1 = 0
    correct_0 = 0
    incorrect_0 = 0
    total_0 = 0
    total_1 = 0
    for i in range(0,len(y_pred)):
        if y_pred[i] == 1:
            total_1 += 1
        else:
            total_0 += 1

        if y_test[i] == 1 and y_test[i] == y_pred[i]:
            correct_1 += 1
        elif y_test[i] == 1 and y_test[i] != y_pred[i]:
            incorrect_1 += 1
        elif y_test[i] == 0 and y_test[i] != y_pred[i]:
            incorrect_0 += 1
        else:
            correct_0 += 1
    #print(correct_1, incorrect_1, incorrect_0, correct_0)
    if total_0 != 0 and total_1 != 0:
        per_1 = (correct_1/total_1)*100
        per_0 = (correct_0/total_0)*100
    else:
        per_1 = 0
        per_0 = 0

    return per_1, total_1, per_0, total_0

a11, b11, c11, d11 = strat3(2, 10) #k = 2, w = 10 
print(a11, b11, c11, d11)
#strat2(2, 10)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

83.33333333333334 6 57.14285714285714 14


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [19]:
output_file_name = os.path.join(file_dir, 'Classifier_Performance.csv')
with open(output_file_name, mode='w') as output_file:
    output_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    output_writer.writerow(['Classifier', '%pct correct for ‘+','Number of + labels','%pct correct for ‘-','Number of - labels'])
    
    output_writer.writerow(['Logistic Regression', a1, b1, c1, d1])
    output_writer.writerow(['KNN (k=5)', a2, b2, c2, d2])
    output_writer.writerow(['Naïve Bayesian', a3, b3, c3, d3])
    output_writer.writerow(['k-means (k=3)', a4, b4, c4, d4])
    output_writer.writerow(['Linear Regression', a5, b5, c5, d5])
    output_writer.writerow(['Decision Tree (Entropy)', a6, b6, c6, d6])
    output_writer.writerow(['SVM (Linear)', a7, b7, c7, d7])
    output_writer.writerow(['SVM (Radial basis function)', a8, b8, c8, d8])
    output_writer.writerow(['Naive Stratergy', a9, b9, c9, d9])
    output_writer.writerow(['Moving Average', a10, b10, c10, d10])
    output_writer.writerow(['Bollinger Band', a11, b11, c11, d11])